In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import kagglehub
from pathlib import Path

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
# import
path = Path(kagglehub.dataset_download("ahmedabbas757/coffee-sales"))/"Coffee Shop Sales.xlsx"
raw_records = pd.read_excel(path, index_col="transaction_id")

In [ ]:
# transform
transaction_datetime = pd.to_datetime(raw_records["transaction_date"].astype(str) + " " + raw_records["transaction_time"].astype(str))
transaction_datetime = pd.Series(transaction_datetime, name="transaction_datetime",index=None)

# dropped columns: store_id, transaction_date, transaction_time, product_id
records = raw_records.reindex([
    "transaction_datetime", 'transaction_qty', 
    'store_location', 'unit_price', 
    'product_category', 'product_type', 'product_detail'], axis="columns")

records["transaction_datetime"] = transaction_datetime
records.sort_values("transaction_datetime", inplace=True)

columns = ["store_location","product_category","product_type", "product_detail"]
records[columns] = records.loc[:,columns].astype("category")

records.head()

---
**How does the average monthly transaction quantity vary across different store locations,\
and which location exhibits the highest variance in sales volume?**

In [ ]:
agg_data = (records
            .groupby(["store_location",records["transaction_datetime"].dt.month], observed=True)["transaction_qty"]
            .mean()
            .unstack()
            .round(2)
            .T
           )
agg_data.index = pd.Index(["Jan", "Feb", "Mar", "Apr", "May", "Jun"],name="month")

In [ ]:
# charting
ax = plt.subplot()
ax.plot(agg_data, label=agg_data.columns,marker=".")
ax.legend()
ax.set_ylim(ax.get_ylim()[0]*0.99,ax.get_ylim()[1]*1.01)
ax.set_title("Average transaction quantity of each store")
ax.set_xlabel("Month")
ax.set_ylabel("Transaction quantity")
agg_data.agg("var")
# The transaction quantities show close to no change over the months and all stay in a very small band (small variance)

---
**What is the median unit price for each product category,\
and how do the interquartile ranges compare across categories?**

In [ ]:
quartiles = records.groupby("product_category",observed=True)["unit_price"].quantile([0.25,0.5,0.75]).unstack()

print("Median unit price for each product category:")
quartiles[0.5]

In [ ]:
print("Interquantile ranges compared:")
(quartiles[0.75] - quartiles[0.25]).sort_values()
# some products exibit no price variation accross types and over time (flavours), while other (branded) have a wide price range

---
**Are there any outliers in the transaction quantities based on the IQR method?**

In [ ]:
groups = records.groupby("product_category", observed=True, as_index=False)["transaction_qty"]
data = {}

for category, srs in groups:
    data.update({category:srs.array})

In [ ]:
# charting
fig, ax = plt.subplots()
ax.boxplot(data.values(), orientation="horizontal", tick_labels=data.keys(), sym="*")
ax.set_title("Distribution of transaction quantities")
ax.set_xlim(0)
ax.set_xlabel("Quantity")
ax.set_ylabel("Category");

---
**What is the distribution of total sales per hour of the day,\
and at what time do stores experience the highest and lowest median sales?**

In [ ]:
revenue = pd.Series(records["unit_price"] * records["transaction_qty"],name="revenue")
revenue.index = pd.Index(records["transaction_datetime"])

In [ ]:
mean_revenue = (revenue
                .groupby([revenue.index.date, revenue.index.hour])
                .sum()
                .unstack()
                .median(axis="rows")
               )

In [ ]:
mean_revenue

In [ ]:
# charting
ax = plt.subplot()
ax.plot(mean_revenue,color="green",marker=".")
ax.set_xticks(mean_revenue.index)
ax.grid()
ax.set_title("Median hourly revenue")
ax.set_ylabel("Median revenue")
ax.set_yticks(range(
    int(mean_revenue.min()), int(mean_revenue.max()*1.1), 50
))
ax.set_xlabel("Hour of the day");

---
**How do sales fluctuate by day of the week,\
and which weekday has the highest standard deviation in transaction quantities?**

In [ ]:
weeks = records["transaction_datetime"].dt.isocalendar()["week"].to_numpy()
# correcting the first week of the year from 52 to 1
weeks = np.where(weeks==52,1,weeks)
days = records["transaction_datetime"].dt.weekday.to_numpy()

daily_revenue = revenue.groupby([weeks, days]).sum().unstack()
daily_revenue.index.name = "week"
daily_revenue.columns.name = "day"

daily_deviation = daily_revenue.std(axis="rows")
daily_median_revenue = daily_revenue.median(axis="rows")

In [ ]:
# charting
fig, (line, bar) = plt.subplots(1,2,sharex=True, figsize=(19,5))

# line setting
line.plot(daily_median_revenue,color="green")
line.set_xticks(daily_median_revenue.index)
line.set_xticklabels(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"],rotation=30)
line.grid()
line.set_title("Median daily revenue")
line.set_xlabel("Day of the week")
line.set_ylabel("Median revenue")

# bar setting
bar.bar(daily_deviation.index, daily_deviation.array, width=0.5,color="green")
bar.grid(axis="y")
bar.set_xticks(daily_deviation.index)
bar.set_xticklabels(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"],rotation=30)
bar.set_ylim(
    daily_deviation.min()*0.7, daily_deviation.max()*1.05
)
bar.set_yticks(
    range(int(bar.get_ylim()[0]), int(bar.get_ylim()[1]), 50)
)
bar.set_title("Daily standard deviation of the revenue")
bar.set_xlabel("Day of the week")
bar.set_ylabel("Standard deviation");

---
**What is the trend in total revenue over time—does it exhibit any seasonal patterns or anomalies?**

In [ ]:
# todo: make weeks dynamic
grouped_revenue = revenue.groupby(weeks).sum()

first_week_of_month = (records
                       .groupby(records["transaction_datetime"].dt.month)
                       .first()["transaction_datetime"]
                       .dt
                       .isocalendar()["week"]
                       .to_numpy()
                      )
first_week_of_month = np.where(first_week_of_month==52,1,first_week_of_month)
first_week_of_month = pd.Series(["January","February", "March", "April", "May", "June"],index=first_week_of_month)

In [ ]:
# charting
legend = False
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(grouped_revenue,color="green",marker=".",label="Revenue")
ax.set_xticks(grouped_revenue.index)
ax.set_xlim(weeks[0]-0.2,weeks[-1]+0.5)
ax.set_title("Weekly revenue over time")
ax.set_xlabel("Week of the year")
ax.set_ylabel("Revenue")
ax.grid()

for week, month_name in first_week_of_month.items():
    ax.axvline(
        x=week,
        color="blue",
        linestyle="dotted",
        label="Month break"
    )

    ax.text(
        x=week+0.1,
        y=40000,
        s=month_name,
        color="blue"
    )

    if not legend:
        ax.legend()
        legend = True

---
**What is the coefficient of variation (CV) in transaction quantities across different store locations,\
and which store shows the most consistent sales performance?**

In [ ]:
stddev = records.groupby("store_location",observed=True)["transaction_qty"].std()
mean = records.groupby("store_location",observed=True)["transaction_qty"].mean()

cv_coefficient = stddev / mean

In [ ]:
# charting
ax = plt.subplot()
ax.bar(
    x=cv_coefficient.index, 
    height=cv_coefficient.to_numpy(),
    width=0.3,
    color="green"
)
ax.set_ylim(0,cv_coefficient.max()*1.3)
ax.set_yticks(
    np.arange(0,cv_coefficient.max()*1.3,0.05)
)
ax.set_title("Coefficient of variation (CV) of the transaction quantity across the stores")
ax.set_xlabel("Store")
ax.set_ylabel("Coefficient of variation")
ax.grid(axis="y")

---
**How does the total revenue per store compare,\
and which store is the top performer in terms of both mean and median monthly revenue?**

In [ ]:
def get_monthly_revenue(df:pd.DataFrame) -> pd.Series:
    rev = df["transaction_qty"] * df["unit_price"]
    return rev.groupby(df["transaction_datetime"].dt.month).sum()
    
total_monthly_revenue = (records
                        .groupby("store_location",observed=True)
                        .apply(get_monthly_revenue,include_groups=False)
                        .T
                       )

total_revenue = total_monthly_revenue.sum(axis="rows")

In [ ]:
# charting
fig, (box, bar) = plt.subplots(ncols=2,figsize=(15,5))

# boxplots
axes = box.boxplot(
    x=total_monthly_revenue,
    orientation="horizontal",
    tick_labels=total_monthly_revenue.columns,
    meanline=True,
    showmeans=True
)
box.legend([axes["means"][0], axes["medians"][0]], ["mean","median"])
box.set_title("Distribution of monthly revenue")
box.set_ylabel("Store")
box.set_xlabel("Monthly revenue")

# bars
bar.bar(
    x=total_revenue.index,
    height=total_revenue.array,
    width=0.3,
    color="green"
)
bar.set_title("Total revenue to date")
bar.set_xlabel("Store")
bar.set_ylabel("Revenue");
bar.set_ylim(total_revenue.min()*0.8,total_revenue.max()*1.05);
bar.grid(axis="y");

---
**Which store has the highest average transaction value,\
and is there a significant difference between the top and bottom performers?**

In [ ]:
mean_trans_value = records.groupby("store_location",observed=True).apply(
    lambda df: (df["unit_price"] * df["transaction_qty"]).mean(),
    include_groups=False
)

In [ ]:
# charting
ax = plt.subplot()
ax.bar(
    x=mean_trans_value.index,
    height=mean_trans_value.array,
    width=0.3,
    color="green"
)
ax.set_title("Mean transaction values across the stores")
ax.set_xlabel("Store")
ax.set_ylabel("Mean transaction value")
ax.set_ylim(mean_trans_value.min()*0.8,mean_trans_value.max()*1.05);
ax.grid(axis="y");

---
**What is the most frequently purchased product type,\
and how does its sales distribution compare to the least frequently purchased one?**

In [ ]:
product_sales = records["product_category"].value_counts()

In [ ]:
# charting
ax = plt.subplot()
ax.bar(
    x=product_sales.index,
    height=product_sales.array,
    width=0.3,
    color="green",
)
ax.set_title("Sales numbers of every product category")
ax.set_xlabel("Product category")
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_ylabel("Absolute number of sales")
ax.set_ylim(product_sales.min()*0.8,product_sales.max()*1.05);
ax.grid(axis="y")

ax2 = ax.twinx();
ax2.set_yticks(ax.get_yticks())
ax2.set_ylim(*ax.get_ylim())
ax2.set_yticklabels(
    (ax2.get_yticks()/len(records)).round(4)
);
ax2.set_ylabel("Relative frequency of the category");

---
**Which product category has the highest price variability,\
and what does the standard deviation suggest about price dispersion?**

In [ ]:
categories_cv = (records
                 .groupby("product_category",observed=True)["unit_price"]
                 .apply(lambda srs: srs.std() / srs.mean())
                ).sort_values(ascending=False)

In [ ]:
# charting
ax = plt.subplot()
ax.bar(
    x=categories_cv.index,
    height=categories_cv.array,
    width=0.3,
    color="green",
    #orientation="horizontal"
)
ax.set_title("Coeffient of variance of price for each product category")
ax.set_xlabel("Product category")
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_ylabel("Coeffient of variance")
ax.set_ylim(categories_cv.min()*0.8,categories_cv.max()*1.1)
ax.set_yticks(np.arange(0,categories_cv.max()*1.05,0.05))
ax.grid(axis="y");

---
**Is there a significant difference in average unit price 
between different product types within the same product category?**

In [ ]:
data = {}
categories = records["product_category"].cat.categories
for category, indices in records.groupby("product_category", observed=True)["unit_price"].groups.items():
    data.update({category: records.loc[indices,"unit_price"].to_numpy()})

In [ ]:
# charting
ax = plt.subplot()
ax.boxplot(
    x=data.values(),
    label=data.keys(),
    orientation="horizontal"
);
ax.set_yticklabels(data.keys())
ax.set_title("Unit price distribution of each product category")
ax.set_ylabel("Product category")
ax.set_xlabel("Unit price")
ax.set_xticks(range(0,50,5));

---
**What percentage of transactions fall into the top 10% of transaction values,\
and how does this compare across store locations?**

In [ ]:
# 90% quantile of tansaction values = values below which 90% of transactions lie = 
# values above which 10% of transactions lie
transaction_values = records["transaction_qty"] * records["unit_price"]
grouped_transaction_values = transaction_values.groupby(records["store_location"],observed=True)
store_quantiles = grouped_transaction_values.quantile(0.9)

top_percentage = grouped_transaction_values.apply(
    lambda df: (df >= store_quantiles.loc[df.name]).value_counts(normalize=True).loc[True].round(4)
)

pd.DataFrame({
    "90th quantile (Dollar amount)":store_quantiles,
    "Percentage of transactions in the top 10%":top_percentage
})